<a href="https://colab.research.google.com/github/paulbonin/RLclass_MVA/blob/main/Final_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This homework is due on April 3rd. Please upload a PDF file describing your investigations and the answers to the different questions (1. to 7.) here : https://nextcloud.univ-lille.fr/index.php/s/gdTY9KqEB2BADwL

Please name your file NAME_FirstName.pdf

# Exploration-Exploitation in Reinforcement Learning

In this homework, you will implement the **UCBVI** algorithm, for exploration in MDPs with finite states and actions and a **finite horizon** criterion. In a finite horizon criterion, the value function of a policy (starting from the $h$ step of the episode) is

$$V_h^{\pi}(s) = \mathbb{E}^{\pi}\left[\left.\sum_{\ell = h}^{H} \gamma^{\ell-h} r_{\ell} \right| s_h = s\right]$$

where the discount parameter $\gamma \in (0,1]$ is often set to $\gamma = 1$.

### Useful libraries

We will need a some tabular RL environments that are implement in the rlberry-scool library, that is developped within the Inria team Scool. You can  install it below (in a conda env if you do it locally).

In [ ]:
!pip install rlberry-scool

In [ ]:
# other useful imports
import numpy as np
import numba
import matplotlib.pyplot as plt
from copy import deepcopy
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from IPython.display import Video, Image
from rlberry_scool.envs.finite import get_discrete_mountain_car_env


## Environment

Our goal is to learn a good policy in a [Mountain Car](https://gymnasium.farama.org/environments/classic_control/mountain_car/) environment. The Mountain Car environement as implemented in gymnasium has a continuous state space. In order to apply UCBVI, we will use a discretized version from the rlberry library.

In [ ]:
def record_video(env, name, horizon=180,policy=None):
    """
    input
    horizon : length of the simulation
    policy : either a determinstic policy represented by an (H,S) array
    or a random policy which is uniform (None)
    """
    env = deepcopy(env)
    env = RecordVideo(env, "./gym_videos/"+name)
    s, _ = env.reset()
    done = False
    tot_reward = 0
    h = 0
    while not done:
        if policy is not None:
            action = policy[h, s]
        else:
            action = env.action_space.sample()
        s, r, term, trunc, infos = env.step(action)
        h += 1
        tot_reward += r
        done = (term or trunc) or h >= horizon
    env.close()
    print("Reward sum: {}".format(tot_reward))


In [ ]:
class ScaleRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        next_state, reward, term, trunc, info = self.env.step(action)
        if term:
            reward = 1.0
        else:
            reward = 0.0

        return (next_state,reward,term,trunc,info)

With the reward wrapper above, the agent gets a reward 0 in states that are not the top of the right hill. Once the goal state is reached, the agent stays in this state and gets a reward 1 until the end of the episode.

In [ ]:
env = ScaleRewardWrapper(get_discrete_mountain_car_env())
record_video(env, name="Random-MountainCar") #saves the video

In [ ]:
X = env.discretized_states[0,:] # discretized positions
Xdot = env.discretized_states[1,:] # discretized velocities

test = 67

print(env.observation_space)
print(env.action_space)
print("state ",test," is ", (X[test],Xdot[test]))
Video("gym_videos/Random-MountainCar/rl-video-episode-0.mp4", embed=True)

# Implementation of backward induction (i.e. value iteration)

In a finite-horizon MDP, the optimal Bellman equations given a recursion that can be used to compute the optimal value function. We have $V_{H+1}^\star(s) = 0$ for all $s$ and for $h \leq H$,

$$Q^\star_h(s,a) = r(s,a) + \gamma \sum_{s\prime \in \mathcal{S}} p(s\prime|s,a) V^\star_{h+1}(s\prime) \ \ \text{and } \ \ V^\star_{h}(s) = \max_{a \in \mathcal{A}} Q_h^\star(s,a).$$

Recall that the optimal policy is deterministic but *non-stationary* and satisfies $\pi^\star_h(s) = \text{argmax}_{a} Q^\star_h(s,a)$.

**Complete the code below in order to compute the optimal Q function in a finite-horizon MDP.**

Note that this code will also be useful to compute the policy used in each episode by UCB-VI, where we have to compute the optimal policy in an optimistic MDP.

In [ ]:
@numba.jit(nopython=True)  # use this to make the code much faster!

def backward_induction(P, R, H, gamma=1.0):
    """
    Parameters:
        P: transition function (S,A,S)-dim matrix
        R: reward function (S,A)-dim matrix
        H: horizon
        gamma: discount factor. Usually set to 1.0 in finite-horizon problems

    Returns:
        The optimal Q-function: array of shape (horizon, S,A)
    """
    S, A = P.shape[0], P.shape[1]
    V = np.zeros((H + 1, S))
    Q = np.zeros((H+1,S,A))
    for h in range(H-1, -1, -1):
        for s in range(S):
            ## TO BE COMPLETED WITH THE Q-VALUE COMPUTATION
            # compute the value
            V[h,s] = np.max(Q[h,s,:])
            # ... and clip it (needed later in UCB-VI)
            if (V[h, s] > H - h):
                V[h, s] = H - h
    return Q

You cannot try this function on the moutain car environement, as the expected rewards and transition probabilities cannot be easily computed, and are not embedded in the environment.

However, you may check it on a simpler Gridworld environment, as P and R are properties of the environment.

In [ ]:
# Testing the implementation in a GridWorld
from rlberry_scool.envs.finite import GridWorld

test_env = GridWorld(nrows=8, ncols=8)
test_env.renderer_type = "pygame"
H = 50 # pick an horizon which is sufficient to reach the goal

Q_test = backward_induction(test_env.P,test_env.R,H,gamma=1.0)

state, _ = test_env.reset()
test_env.enable_rendering()
for h in range(H):
  action = np.argmax(Q_test[h, state])
  next_state, reward, term, trunc, info = test_env.step(action)
  if term:
    break
  state = next_state

# save video (run next cell to visualize it)
test_env.save_gif('gridworld_backward_induction.gif')
# clear rendering data
test_env.clear_render_buffer()
test_env.disable_rendering()


In [ ]:
Image(open('gridworld_backward_induction.gif','rb').read())

# Implementation of UCBVI

The UCBVI algorithm works as follows:

* In each episode $t$, the agent has observed $n_t$ transitions $(s_i, a_i, r_i, s_{i+1})_{i=1}^{n_t}$ of states, actions, rewards and next states.
* We estimate a model of the MDP as:
$$
\mathbf{rewards:}\quad\widehat{R}_{t(s, a)} = \frac{1}{N_t(s, a)} \sum_{i=1}^{n_t} \mathbb{1}\{s = s_i, a = a_i\} r_i
\\
\mathbf{transitions:}\quad \widehat{P}_t(s\prime|s, a) =  \frac{1}{N_t(s, a)} \sum_{i=1}^{n_t} \mathbb{1}\{s = s_i, a = a_i, s\prime=s_{i+1}\}
$$
where
$$
N_t(s, a) = \max\left(1, \sum_{i=1}^{n_t} \mathbb{1}\{s = s_i, a = a_i\} \right)
$$
* We define an appropriate exploration bonus, which is some function of the number of visits, for example
$$
B_t(s, a) \propto \sqrt{\frac{1}{N_t(s, a)}} \cdot
$$

* Then, in episode $t$, we compute $\{Q_h^t(s, a)\}_{h=1}^H$ as the ($H$-horizon) optimal value functions in the MDP whose transitions are $\widehat{P}_t$ and whose rewards are $(\widehat{R}_t + B_t)$. At step $h$ of episode $t$, the agent chooses the action $a_h^t \in \arg\max_a Q_h^t(s, a)$.

**Complete the code below to implement UCBVI.**

In [ ]:
# An example of bonus function
def bonus(N):
    """input : a numpy array (nb of visits)
    output : a numpy array (bonuses)"""
    nn = np.maximum(N, 1)
    return np.sqrt(1.0/nn)

# The UCB-VI algorithm
def UCBVI(env,H, nb_episodes,verbose="off",bonus_function=bonus,gamma=1):
    """
    Parameters:
        env: environement
        bonus_function : maps the number of visits to the corresponding bonus
        H: horizon
        gamma: discount factor. Usually set to 1.0 in finite-horizon problems

    Returns:
        episode_rewards: a vector storing the sum of rewards obtained in each episode
        states_visited: a vector storing the number of states/action pairs visited until each episode
        N_sa : array of size (S,A) giving the total number of visits in each state
        Rhat : array of size (S,A) giving the estimated average rewards
        Phat : array of size (S,A,S) giving the estimated transition probabilities
        optimistic_Q : array of size (H,S,A) giving the optimistic Q function used in the last episode
    """
    S = env.observation_space.n
    A = env.action_space.n
    Phat = np.ones((S,A,S)) / S
    Rhat = np.zeros((S,A))
    N_sa = np.zeros((S,A), dtype=int) # number of visits

    N_sas = np.zeros((S,A,S), dtype=int) # number of transitions
    S_sa = np.zeros((S,A)) # cumulative rewards
    episode_rewards = np.zeros((nb_episodes,))
    states_visited = np.zeros((nb_episodes,))


    for k in range(nb_episodes):
        sum_rewards = 0
        ### TO BE COMPLETED: RUN AN EPISODE OF UCV-VI

        # update sum of rewards and number of visits
        episode_rewards[k] = sum_rewards
        states_visited[k] = (N_sa.sum(axis=1) > 0).sum()

        if (verbose=="on"):
          # periodically display the rewards collected and visited states
          if k % 50 == 0 or k==1:
            print("rewards in episode {}: {}".format(k, episode_rewards[k]), end = ", ")
            print("Number of visited states: ", states_visited[k] )
            # print(V[0, :])

    return episode_rewards, states_visited, N_sa, Rhat, Phat,optimistic_Q

**1. To check whether the algorithm is working, you can visualize the amount of rewards gathered in episodes with the default bonus, as well as the number of visited states since the beginning (which measures how well the environment is being explored). Based on your findings, how many episodes seem necessary for the algorithm to behave well?**

In [ ]:
HORIZON = 200
NUM_EPISODES = 500

env = ScaleRewardWrapper(get_discrete_mountain_car_env())
rewards, cum_visits,N_sa, Rhat, Phat, optimistic_Q = UCBVI(env, H=HORIZON, nb_episodes=NUM_EPISODES,verbose="on")

In [ ]:
plt.plot([np.cumsum(rewards)[episode]/(episode+1) for episode in range(NUM_EPISODES)])
#plt.plot(rewards)
plt.title("average reward per episode")

plt.figure(2)
plt.plot(cum_visits)
plt.title("number of states visited since the first episode")

**2. UCBVI's first purpose is not to output a candidate optimal policy, but if you want to do so, what is (are) reasonnable candidate(s)? You can check by looking how the agent behaves under this (these) policies, i.e. whether it solves the task. You can also display a 2D visualization of associated Q functions.**

In [ ]:
policy = ## choose the policy to visualize
record_video(env,"UCB-VI-optimistic", HORIZON,policy)
Video('gym_videos/UCB-VI-optimistic/rl-video-episode-0.mp4', embed=True)

Displaying the value of this policy is not possible, as it would require to perform policy evaluation, and as P and R are unknown, Monte-Carlo evaluation is the only option. Let's keep this for later. For now a proxy is to visualize $\max_{a} Q(s,a)$ when $\pi$ is greedy wrt to $Q$. For the optimistic policy, $Q$ is the optimistic Q function.


In [ ]:
Q_function = ## here goes Q such that policy =greedy(Q)
value = np.max(Q-function,axis=2)
plt.scatter(env.discretized_states[0, :], env.discretized_states[1, :], c=value[0, :], s=400)
plt.xlabel('Car Position')
plt.ylabel('Car Velocity')
clb=plt.colorbar()
clb.ax.set_title('Value')
plt.show()

In theory, any algorithm with an upper bound $\mathbb{E}[R_T] \leq B_T$ on its expected regret can be used to output an $\varepsilon$-optimal policy with high probability using the following trick: run the algorithm for a sufficiently large $T$ and output a policy $\widehat{\pi}_T$ chosen at random among the first $T$ policies used by the algorithm. The point of this question is to quantify how long.    

**3. [Theory] For a given $T$, give an upper bound on the probability $\mathbb{P}(V^\star - V^{\widehat{\pi}_T} > \varepsilon)$ that depends on $T,\varepsilon$ and $B_T$ (you can use Markov's inequality). Deduce the order of magnitude of the number of episodes needed by UCB-VI to output a $\varepsilon$-optimal policy with probability $1-\delta$. Comment on the result.**

**4. UCB-VI has actually some guarantees in terms of regret, so you can try to optimize the algorithm for the setting it is designed for.**

Explore different kind of bonusses for UCB-VI, in terms of resulting *expected* cumulative rewards. You may also compare bonus-based "directed" exploration to a model-based algorithm using instead $\epsilon$-greedy exploration.

**5. [Theory] Our goal was to find a policy that manages to get the car up the hill. We chose to model this as solving an episodic MDP with a large enough horizon $H$. Could this task also be modelled as solving a discounted MDP? What value would you choose for the discount?**

**6. Compare the best model-based algorithm found in question 4. to a model-free alternative (still in terms of cumulative rewards).**

The idea is to try to make some variant of the (discounted) Q-Learning algorithm work for this problem. Learning will however still proceed in episodes of length H, with a reset, and we will similarly monitor the (undiscounted) amount of rewards gathered in each episode, for comparison with UCBVI.

You may explore one of the following possibilities: combining $\varepsilon$-greedy exploration with a careful initialization of the Q-values, or adding some exploration bonuses to the rewards.

Careful describe the chosen algorithm as well as the results.


**7. [Theory] We would now like to compare the sample complexity of the model-free algorithm to that of the model-based algorithm. How would you proceed to do that?**


(you are not required to actually do it, just specify how you would proceed given the time)